In [1]:
## Import packages here
import pandas as pd
import numpy as np 
import imblearn
from matplotlib.pyplot import figure
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from collections import Counter

In [2]:
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# # Installing packages
# !pip install duckdb
# !pip install watermark
# !pip install pysmiles
# !pip install rdkit

In [4]:
# # Database
# import duckdb
# # Database - Parquet format
# data_train = '/kaggle/input/leash-BELKA/train.parquet'
# test_path = '/kaggle/input/leash-BELKA/test.parquet'

# # Bank connection
# con = duckdb.connect()

# # Query
# train = con.query(f"""(SELECT * FROM parquet_scan('{data_train}') 
# WHERE binds = 0
# ORDER BY random()
# LIMIT 5000000)
# UNION ALL
# (SELECT * FROM parquet_scan('{data_train}')
# WHERE binds = 1
# ORDER BY random()
# LIMIT 500000)""").df()

# # Closing database
# con.close()

# # Saving dataset
# train.to_csv("/kaggle/working/dataset.csv")

In [5]:
# # Viewing first 5 data
# #train_ = pd.read_csv('/kaggle/input/leash-BELKA/train.csv')
# test = pd.read_csv('/kaggle/input/leash-BELKA/test.csv')
# train.head()

In [6]:
#check the numbers of samples and features
# print("The train data size before dropping Id feature is : {} ".format(train.shape))
# print("The test data size before dropping Id feature is : {} ".format(test.shape))

In [14]:
# # Shuffle the rows
# train = train.sample(frac=1).reset_index(drop=True)

In [15]:
# train_copy=train.copy()

In [7]:
# Visualize the count for each class
# train_copy['binds'].value_counts().plot.bar(color=['green', 'red'])

In [9]:
# train_copy.isnull().sum()

In [8]:
# test.isnull().sum()

In [19]:
# Get a Pd.Series consisting of all the string categoricals
#one_hot_encode_cols = data_copy.dtypes[data_copy.dtypes == 'object']  # filtering by string categoricals
#one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

#data_copy[one_hot_encode_cols].head().T

In [20]:
# Do the one hot encoding
#data_copy = pd.get_dummies(data_copy, columns=one_hot_encode_cols, drop_first=True)
#data_copy.shape

In [10]:
#join the test and trainset to ensure uniform preprocessing
# ntrain = train_copy.shape[0]
# ntest = test.shape[0]
# y = train_copy[['binds']]
# all_data = pd.concat((train_copy, test)).reset_index(drop=True)
# all_data.drop(['binds'], axis=1, inplace=True)
# print("all_data size is : {}".format(all_data.shape))

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Create a LabelEncoder instance
# le = LabelEncoder()

# # Fit the encoder to the entire dataset (flattened)
# le.fit(all_data[['buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles', 'protein_name', 'molecule_smiles']].values.reshape(-1, 1))

# # Transform each column using the same encoder
# all_data['buildingblock1_smiles'] = le.transform(all_data['buildingblock1_smiles'])
# all_data['buildingblock2_smiles'] = le.transform(all_data['buildingblock2_smiles'])
# all_data['buildingblock3_smiles'] = le.transform(all_data['buildingblock3_smiles'])
# all_data['protein_name'] = le.transform(all_data['protein_name'])
# all_data['molecule_smiles'] = le.transform(all_data['molecule_smiles'])

In [11]:
# all_data.head()

In [23]:
# import seaborn as sns
#all_data.drop('Unnamed: 0', axis=1, inplace= True)
#sns.pairplot(data_copy, plot_kws=dict(alpha=.1, edgecolor='none'))

In [12]:
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Descriptors
# from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
# from sklearn.preprocessing import StandardScaler

# # Calculate molecular descriptors for building blocks and molecule
# def calculate_descriptors(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     descriptors = []
#     descriptors.append(Descriptors.MolWt(mol))
#     #descriptors.append(Descriptors.LogP(mol))
#     descriptors.append(Descriptors.TPSA(mol))
#     descriptors.append(Descriptors.NumHDonors(mol))
#     descriptors.append(Descriptors.NumHAcceptors(mol))
#     return descriptors

# all_data['buildingblock1_descriptors'] = all_data['buildingblock1_smiles'].apply(lambda x: calculate_descriptors(x))
# all_data['buildingblock2_descriptors'] = all_data['buildingblock2_smiles'].apply(lambda x: calculate_descriptors(x))
# all_data['buildingblock3_descriptors'] = all_data['buildingblock3_smiles'].apply(lambda x: calculate_descriptors(x))
# all_data['molecule_descriptors'] = all_data['molecule_smiles'].apply(lambda x: calculate_descriptors(x))
# # Generate molecular fingerprints for building blocks and molecule
# def generate_fingerprints(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     fp = GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
#     return fp

# all_data['buildingblock1_fingerprints'] = all_data['buildingblock1_smiles'].apply(lambda x: generate_fingerprints(x))
# all_data['buildingblock2_fingerprints'] = all_data['buildingblock2_smiles'].apply(lambda x: generate_fingerprints(x))
# all_data['buildingblock3_fingerprints'] = all_data['buildingblock3_smiles'].apply(lambda x: generate_fingerprints(x))
# all_data['molecule_fingerprints'] = all_data['molecule_smiles'].apply(lambda x: generate_fingerprints(x))


In [13]:
# # One-hot encode protein names
# protein_names = all_data['protein_name'].unique()
# all_data['protein_name'] = df['protein_name'].apply(lambda x: protein_names.tolist().index(x))

# # Scale the features using StandardScaler
# scaler = StandardScaler()
# all_data['scaled_buildingblock1_descriptors'] = scaler.fit_transform(all_data['buildingblock1_descriptors'])
# all_data['scaled_buildingblock2_descriptors'] = scaler.fit_transform(all_data['buildingblock2_descriptors'])
# all_data['scaled_buildingblock3_descriptors'] = scaler.fit_transform(all_data['buildingblock3_descriptors'])
# all_data['scaled_molecule_descriptors'] = scaler.fit_transform(all_data['molecule_descriptors'])
# all_data['scaled_buildingblock1_fingerprints'] = scaler.fit_transform(all_data['buildingblock1_fingerprints'])
# all_data['scaled_buildingblock2_fingerprints'] = scaler.fit_transform(all_data['buildingblock2_fingerprints'])
# all_data['scaled_buildingblock3_fingerprints'] = scaler.fit_transform(all_data['buildingblock3_fingerprints'])
# all_data['scaled_molecule_fingerprints'] = scaler.fit_transform(all_data['molecule_fingerprints'])

In [ ]:
# # One-hot encode protein names
# protein_names = all_data['protein_name'].unique()
# all_data['protein_name'] = df['protein_name'].apply(lambda x: protein_names.tolist().index(x))

In [ ]:
# plt.figure(figsize=(18,18))
# sns.heatmap(all_data.corr(),annot=True,cmap='RdYlGn')
# plt.show()

In [14]:
new_df = pd.read_csv('feature_engineered_fingerprints.csv')

In [15]:
new_df = new_df[['id', 'buildingblock1_smiles',
       'buildingblock2_smiles', 'buildingblock3_smiles', 'molecule_smiles',
       'protein_name', 'buildingblock1_descriptors',
       'buildingblock2_descriptors', 'buildingblock3_descriptors',
       'molecule_descriptors']]

In [16]:
# split the Name column into two columns
new_df[['Molecular_Weight_block1', 
    'Topological_Polar_Surface_Area_block1',
    'Number_of_Rotatable_Bonds_block1','Number_of_Hydrogen_Bond_Donors_block1']] = new_df['buildingblock1_descriptors'].str.split(',', expand=True)

In [17]:
# split the Name column into two columns
new_df[['Molecular_Weight_block2', 
    'Topological_Polar_Surface_Area_block2',
    'Number_of_Rotatable_Bonds_block2','Number_of_Hydrogen_Bond_Donors_block2']] = new_df['buildingblock2_descriptors'].str.split(',', expand=True)

In [18]:
# split the Name column into two columns
new_df[['Molecular_Weight_block3', 
    'Topological_Polar_Surface_Area_block3',
    'Number_of_Rotatable_Bonds_block3','Number_of_Hydrogen_Bond_Donors_block3']] = new_df['buildingblock3_descriptors'].str.split(',', expand=True)

In [19]:
new_df[['Molecular_Weight_molecule', 
    'Topological_Polar_Surface_Area_molecule',
    'Number_of_Rotatable_Bonds_molecule','Number_of_Hydrogen_Bond_Donors_molecule']] = new_df['molecule_descriptors'].str.split(',', expand=True)

MemoryError: 

In [ ]:
new_df.drop(['buildingblock3_descriptors', 'buildingblock2_descriptors', 'buildingblock1_descriptors'], axis= 1, inplace=True)

In [ ]:
train_ = all_data[:ntrain]
test_ = all_data[ntrain:]

In [ ]:
rs=123
# Grid search hyperparameters for a random forest model
def grid_search_rf(X_train, y_train):
    params_grid = { 
        'max_depth': [5, 10, 15, 20], 
        'n_estimators': [25, 50, 100], 
    }
    rf_model = RandomForestClassifier(random_state=rs)
    grid_search = GridSearchCV(estimator = rf_model, param_grid = params_grid, scoring='f1', cv = 2, verbose = 1)
    grid_search.fit(X_train.drop(['id'], axis=1), y_train)
    preds = preds = grid_search.predict(x_test.drop(['id'], axis=1))
    probs = grid_search.predict_proba(x_test.drop(['id'], axis=1))[:, 1]
    submit= grid_search.predict_proba(test_.drop(['id'], axis=1))[:, 1]
    submit_df = pd.DataFrame({'id': test_['id'], 'binds': submit})
    submit_df.to_csv('submit_1_rf.csv')

    
    # Confusion matrix
    cm = confusion_matrix(y_test, preds)
    print('Confusion Matrix - Random Forest:\n', cm)
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, probs)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label='Random Forest')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve - Random Forest')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()
    
    # Classification metrics report
    f1 = f1_score(y_test, preds)
    recall = recall_score(y_test, preds)
    precision = precision_score(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    print('Metrics - Random Forest:')
    print('F1 Score:', f1)
    print('Recall:', recall)
    print('Precision:', precision)
    print('Accuracy:', accuracy)

    # Feature importance
    feature_importances = grid_search.best_estimator_.feature_importances_
    feature_names = X_train.drop(['id'], axis=1).columns
    plt.figure(figsize=(8, 6))
    plt.bar(feature_names, feature_importances)
    plt.title('Feature Importances - Random Forest')
    plt.xlabel('Feature')
    plt.ylabel('Importance')
    plt.xticks(rotation=90)
    plt.show()
    
    

# Grid search hyperparameters for a XGBoost model
def grid_search_xgb(X_train, y_train):
    params_grid = {
        'max_depth': [5, 10, 15, 20],
        'n_estimators': [25, 50, 100],
        'learning_rate': [0.1, 0.5, 1],
    }
    xgb_model = XGBClassifier(random_state=rs)
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=params_grid, scoring='f1', cv=2, verbose=1)
    grid_search.fit(X_train.drop('id', axis=1), y_train)
    preds = grid_search.predict(x_test.drop('id', axis=1))
    probs = grid_search.predict_proba(x_test.drop(['id'], axis=1))[:, 1]
    submit= grid_search.predict_proba(test_.drop(['id'], axis=1))[:, 1]
    submit_df = pd.DataFrame({'id': test_['id'], 'binds': submit})
    submit_df.to_csv('submit_1__xg.csv', index=False)

    # Confusion matrix
    cm = confusion_matrix(y_test, preds)
    print('Confusion Matrix - XGBoost:\n', cm)

    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, probs)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label='XGBoost')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve - XGBoost')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

    # Feature importance
    feature_importances = grid_search.best_estimator_.feature_importances_
    feature_names = X_train.drop('id', axis=1).columns
    plt.figure(figsize=(8, 6))
    plt.bar(feature_names, feature_importances)
    plt.title('Feature Importances - XGBoost')
    plt.xlabel('Feature')
    plt.ylabel('Importance')
    plt.xticks(rotation=90)
    plt.show()

     # Classification metrics report
    f1 = f1_score(y_test, preds)
    recall = recall_score(y_test, preds)
    precision = precision_score(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    print('Metrics - XGBoost:')
    print('F1 Score:', f1)
    print('Recall:', recall)
    print('Precision:', precision)
    print('Accuracy:', accuracy)

In [ ]:
def split_data(df, target):
    return train_test_split(df, target, test_size=0.3, stratify=y, random_state = rs)

In [ ]:
# Split the training and testing dataset
x_train, x_test, y_train, y_test = split_data(train_, y)

In [ ]:
grid_search_rf(x_train, y_train)

In [ ]:
from xgboost import XGBClassifier
grid_search_xgb(x_train, y_train)

In [ ]:
pd.read_csv('submit_1__xg.csv').shape